In [5]:
# Import packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

import numpy as np
import tensorflow as tf

In [25]:
# Set logging
tf.logging.set_verbosity(tf.logging.INFO)

In [6]:
# Download datasets, if needed
IRIS_TRAINING = 'data/iris_training.csv'
IRIS_TRAINING_URL = 'http://download.tensorflow.org/data/iris_training.csv'
IRIS_TEST = 'data/iris_test.csv'
IRIS_TEST_URL = 'http://download.tensorflow.org/data/iris_test.csv'

if not os.path.exists(IRIS_TRAINING):
    if not os.path.exists(IRIS_TRAINING):
        raw = urllib.urlopen(IRIS_TRAINING_URL).read()
        with open(IRIS_TRAINING, "w") as f:
            f.write(raw)

    if not os.path.exists(IRIS_TEST):
        raw = urllib.urlopen(IRIS_TEST_URL).read()
        with open(IRIS_TEST, "w") as f:
            f.write(raw)

In [11]:
# Import datasets
def load_dataset_from_csv(filepath): 
    dataset = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=filepath, target_dtype=np.int, features_dtype=np.float32)
    return dataset

training_set = load_dataset_from_csv(IRIS_TRAINING)
test_set = load_dataset_from_csv(IRIS_TEST)

In [41]:
# Build classifier
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

classifier = tf.contrib.learn.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10, 20, 10],
    n_classes=3,
    model_dir='/tmp/iris_model',
    config=tf.contrib.learn.RunConfig(save_checkpoints_secs=1))

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 1, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1197b8350>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [38]:
# Create input function
def input_fn(dataset):
    x = tf.constant(dataset.data)
    y = tf.constant(dataset.target)
    return x, y

In [44]:
# Create validation monitor
prediction_key = tf.contrib.learn.PredictionKey.CLASSES

def get_metric_spec(metric):
    streaming_metric = 'streaming_{}'.format(metric)
    metric_func = getattr(tf.contrib.metrics, streaming_metric)
    return tf.contrib.learn.MetricSpec(
        metric_fn=metric_func,
        prediction_key=prediction_key)
    
    
validation_metrics = {
    'accuracy': get_metric_spec('accuracy'),
    'precision': get_metric_spec('precision'),
    'recall': get_metric_spec('recall')
}

validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    metrics=validation_metrics,
    early_stopping_metric='loss',
    early_stopping_metric_minimize=True,
    early_stopping_rounds=200)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


In [45]:
# Fit model
classifier.fit(
    input_fn=lambda: input_fn(training_set),
    steps=2000,
    monitors=[validation_monitor])

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 8002 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:loss = 0.024685, step = 8002
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator w

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x1197b8110>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x111d42398>, 'input_layer_min_slice_size': None})

In [27]:
# Evaluate accuracy
accuracy_score = classifier.evaluate(
    input_fn=lambda: input_fn(test_set),
    steps=1
)['accuracy']
print('Accuracy: {0:f}'.format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-04-14-14:48:43
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2017-04-14-14:48:44
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.966667, auc = 0.998333, global_step = 4000, loss = 0.0806242
Accuracy: 0.966667


In [ ]:
# 